In [2]:
!wget https://huggingface.co/datasets/rvl_cdip/resolve/main/data/rvl-cdip.tar.gz

--2025-09-27 03:54:24--  https://huggingface.co/datasets/rvl_cdip/resolve/main/data/rvl-cdip.tar.gz
Resolving huggingface.co (huggingface.co)... 3.165.160.12, 3.165.160.59, 3.165.160.61, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.12|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /datasets/aharley/rvl_cdip/resolve/main/data/rvl-cdip.tar.gz [following]
--2025-09-27 03:54:24--  https://huggingface.co/datasets/aharley/rvl_cdip/resolve/main/data/rvl-cdip.tar.gz
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cas-bridge.xethub.hf.co/xet-bridge-us/6261684d6dae705b25672d0d/4f5c5412de7c3527c5f5233b86c5624a1cb2083ba5fe61a5811fa7f023fc7fca?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Content-Sha256=UNSIGNED-PAYLOAD&X-Amz-Credential=cas%2F20250927%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250927T035424Z&X-Amz-Expires=3600&X-Amz-Signature=11f2c590dda3f4c35fc769357fb8d47c

In [ ]:
!wget https://huggingface.co/datasets/rvl_cdip/resolve/main/data/train.txt
!wget https://huggingface.co/datasets/rvl_cdip/resolve/main/data/test.txt
# !wget https://huggingface.co/datasets/rvl_cdip/resolve/main/data/val.txt

In [6]:
!tar -xf rvl-cdip.tar.gz

^C


In [ ]:
# orignal dataset has 320K train images and 40K test images
# train:test = 8:1
x=16
8000//x, 1000//x

(500, 62)

In [ ]:
import os
from datasets import Dataset, ClassLabel, Image

# https://huggingface.co/datasets/aharley/rvl_cdip/blob/main/rvl_cdip.py
id2label = {
  "0": "letter",
  "1": "form",
  "2": "email",
  "3": "handwritten",
  "4": "advertisement",
  "5": "scientific report",
  "6": "scientific publication",
  "7": "specification",
  "8": "file folder",
  "9": "news article",
  "10": "budget",
  "11": "invoice",
  "12": "presentation",
  "13": "questionnaire",
  "14": "resume",
  "15": "memo"
}
label2id = {v: k for k, v in id2label.items()}


def load_images(ds_file:str, num_images_per_class:int=1, return_ds:bool=False):
  _images = open(ds_file).readlines()
  _images = [x.strip() for x in _images]
  # count images per label
  images_per_label = {}
  class_images = {} # store available images
  _dataset = []

  for img in _images:
    img_path, img_label = img.split()
    img_path = os.path.join('images',img_path)
    if not os.path.exists(img_path):
      continue
    class_images[img_label] = class_images.get(img_label,[]) + [img_path]
    # count images per label
    if img_label in images_per_label:
      images_per_label[img_label] += 1
    else:
      images_per_label[img_label] = 1

  min_image_count = min(images_per_label.values())
  print("minimum image count per class:",min_image_count)
  if min_image_count < num_images_per_class:
    print("some labels have fewer images available")
    print(f"num_images_per_class should be <= {min_image_count}")
    return
  elif not return_ds:
    return

  for i in range(num_images_per_class):
    for label in class_images:
      img_path = class_images[label][i]
      _dataset.append({"image": img_path, "label": int(label)})
  return _dataset

def create_classification_dataset(ds_file:str, num_images_per_class:int):
  _ds = load_images(ds_file, num_images_per_class,return_ds=True)
  temp_ds = Dataset.from_list(_ds)
  temp_ds = temp_ds.shuffle() # important
  temp_ds = temp_ds.cast_column("image",Image(decode=True))
  temp_ds = temp_ds.cast_column("label", 
                                ClassLabel(num_classes=len(id2label),
                                           names=list(id2label.values())
                                           ))
  return temp_ds

In [19]:
load_images('train.txt')
load_images('test.txt')

minimum image count per class: 10621
minimum image count per class: 1295


In [20]:
train_dataset = create_classification_dataset('train.txt', 500)
test_dataset = create_classification_dataset('train.txt', 62)

minimum image count per class: 10621


Casting the dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

minimum image count per class: 10621


Casting the dataset:   0%|          | 0/992 [00:00<?, ? examples/s]

In [21]:
from datasets import DatasetDict

rvl = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
  })
rvl

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 992
    })
})

In [31]:
rvl.push_to_hub("hf-tuner/rvl-cdip-document-classification")

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ? shards/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

...

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :   1%|1         | 1.61MB /  117MB            

CommitInfo(commit_url='https://huggingface.co/datasets/hf-tuner/rvl-cdip-document-classification/commit/c38e401f91b8941239efd8fb5733291a031444cc', commit_message='Upload dataset', commit_description='', oid='c38e401f91b8941239efd8fb5733291a031444cc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/hf-tuner/rvl-cdip-document-classification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='hf-tuner/rvl-cdip-document-classification'), pr_revision=None, pr_num=None)